In [62]:
#Read in the CSV file
import pandas as pd
deposit_data = pd.read_csv('phase1_raw_transactions.csv', parse_dates=["time", "Customer_Initiated_At", "Customer_Credited_At"])

In [64]:
deposit_data['Customer_Initiated_At'].dtype

dtype('<M8[ns]')

In [66]:
deposit_data["week"] = deposit_data["Customer_Initiated_At"].dt.to_period("W").apply(lambda r: r.start_time)



In [67]:
# Group by user and week
weekly_summary = deposit_data.groupby(["uid", "week"]).agg(
    weekly_volume_usd=("amount", "sum"),
    weekly_tx_count=("tx_ref", "count"),
    avg_txn_value=("amount", "mean")
).reset_index()

# Sort (optional but helpful for trend tracking)
weekly_summary.sort_values(by=["uid", "week"], inplace=True)

# Preview the result
weekly_summary.head()


,uid,week,weekly_volume_usd,weekly_tx_count,avg_txn_value
0,user_1,2024-04-29,226.10,3,75.366667
1,user_1,2024-05-06,106.44,3,35.480000
2,user_1,2024-05-13,195.89,4,48.972500
3,user_1,2024-05-20,569.61,3,189.870000
4,user_1,2024-05-27,575.38,5,115.076000


In [68]:
#Calculate week-over-week % change in volume for each user
weekly_summary["pct_change"] = (
    weekly_summary
    .groupby("uid")["weekly_volume_usd"]
    .pct_change()
)


In [69]:
# Step 4: Flag drop-offs (50%+ drop)
weekly_summary["dropoff_flag"] = weekly_summary["pct_change"] < -0.5


In [70]:
# See users with at least one drop-off
weekly_summary[weekly_summary["dropoff_flag"]].head()


,uid,week,weekly_volume_usd,weekly_tx_count,avg_txn_value,pct_change,dropoff_flag
1,user_1,2024-05-06,106.44,3,35.480000,-0.529235,True
6,user_1,2024-06-10,441.94,6,73.656667,-0.675101,True
8,user_1,2024-06-24,366.00,2,183.000000,-0.593550,True
9,user_1,2024-07-01,157.39,3,52.463333,-0.569973,True
11,user_1,2024-07-15,228.04,3,76.013333,-0.834217,True


In [86]:
drop_summary = (
    weekly_summary[weekly_summary["dropoff_flag"]]
    .groupby("week")["uid"]
    .nunique()
    .reset_index(name="users_dropped")
)

In [88]:
drop_summary

,week,users_dropped
0,2024-05-06,427
1,2024-05-13,654
2,2024-05-20,664
3,2024-05-27,643
4,2024-06-03,674
5,2024-06-10,666
6,2024-06-17,638
7,2024-06-24,662
8,2024-07-01,670
9,2024-07-08,632


In [78]:
# Find first week each user dropped off
first_drop = (
    weekly_summary[weekly_summary["dropoff_flag"]]
    .groupby("uid")["week"]
    .min()
    .reset_index(name="first_drop_week")
)

# Merge back into weekly_summary
weekly_summary = weekly_summary.merge(first_drop, on="uid", how="left")


In [90]:
first_drop

,uid,first_drop_week
0,user_1,2024-05-06
1,user_10,2024-06-03
2,user_100,2024-05-13
3,user_1000,2024-06-03
4,user_1001,2024-05-06
...,...,...
2953,user_994,2024-05-20
2954,user_996,2024-07-22
2955,user_997,2024-06-24
2956,user_998,2024-05-20


In [80]:
drop_severity = (
    weekly_summary[weekly_summary["dropoff_flag"]]
    .groupby("uid")["pct_change"]
    .min()
    .reset_index(name="drop_severity")
)

weekly_summary = weekly_summary.merge(drop_severity, on="uid", how="left")


In [92]:
drop_severity

,uid,drop_severity
0,user_1,-0.834217
1,user_10,-0.910875
2,user_100,-0.972917
3,user_1000,-0.999779
4,user_1001,-0.968768
...,...,...
2953,user_994,-0.672626
2954,user_996,-0.839811
2955,user_997,-0.815901
2956,user_998,-0.761508


In [82]:
# Fill NA for dropoff flag to False for clean logic
weekly_summary["dropoff_flag"] = weekly_summary["dropoff_flag"].fillna(False)

# Create cumulative drop tracker
weekly_summary["ever_dropped"] = (
    weekly_summary.groupby("uid")["dropoff_flag"].transform("cummax")
)

# Flag recovery: volume > previous AND has dropped before
weekly_summary["recovered"] = (
    (weekly_summary["pct_change"] > 0) & (weekly_summary["ever_dropped"])
)


In [56]:
if "first_drop_week" in weekly_summary.columns:
    weekly_summary.drop(columns="first_drop_week", inplace=True)

weekly_summary = weekly_summary.merge(first_drop, on="uid", how="left")


In [96]:
weekly_summary

,uid,week,weekly_volume_usd,weekly_tx_count,avg_txn_value,pct_change,dropoff_flag,first_drop_week,drop_severity,ever_dropped,recovered
0,user_1,2024-04-29,226.10,3,75.366667,NaN,False,2024-05-06,-0.834217,False,False
1,user_1,2024-05-06,106.44,3,35.480000,-0.529235,True,2024-05-06,-0.834217,True,False
2,user_1,2024-05-13,195.89,4,48.972500,0.840380,False,2024-05-06,-0.834217,True,True
3,user_1,2024-05-20,569.61,3,189.870000,1.907805,False,2024-05-06,-0.834217,True,True
4,user_1,2024-05-27,575.38,5,115.076000,0.010130,False,2024-05-06,-0.834217,True,True
...,...,...,...,...,...,...,...,...,...,...,...
38304,user_999,2024-06-17,535.28,5,107.056000,0.535029,False,2024-05-20,-0.923758,True,True
38305,user_999,2024-06-24,246.65,3,82.216667,-0.539213,True,2024-05-20,-0.923758,True,False
38306,user_999,2024-07-01,203.46,5,40.692000,-0.175106,False,2024-05-20,-0.923758,True,False
38307,user_999,2024-07-08,442.23,6,73.705000,1.173548,False,2024-05-20,-0.923758,True,True


In [100]:
weekly_summary.to_csv("weekly_user_volume_summary.csv", index=False)


In [112]:

total_recovered = weekly_summary[weekly_summary["recovered"]].uid.nunique()
total_dropped = weekly_summary[weekly_summary["dropoff_flag"]].uid.nunique()
recovery_rate = total_recovered / total_dropped


In [114]:
total_recovered

2852